import pandas as pd
import numpy as np


df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

(df[pd.isna(df["region_1"]) & pd.isna(df["region_2"])])

df.info()

df

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(df["designation"].value_counts())

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df["designation"].value_counts()[:100])

#df[df["designation_tr"] == "reserva" | df["designation_tr"] == "riserva" | df["designation_tr"] == "rèserve"]["designation_tr"] = "reserve"
df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])


df

# rimuovo regioni doppie e creo regione unica
df.loc[df["region_1"] == df["region_2"], "region_2"] = ""
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)

singular_region_complete = set(df["region_complete"])
len(singular_region_complete)

import geocoder

g = geocoder.arcgis('Redlands, CA')
print(g.latlng)

import geocoder

singular_region_complete_with_latlong = []
for region in singular_region_complete:
    print(region)
    loc =  geocoder.arcgis(region,maxRows=1)
    print(loc)
    try:
        singular_region_complete_with_latlong.append((region, loc.latlng[0],loc.latlng[1]))   
    except:
        pass

with open('region_coordinates.csv', 'w') as fp:
    fp.write('\n'.join('%s,%f,%f' % x for x in singular_region_complete_with_latlong))

df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, how="left", on="region_complete")

df.to_csv("competition_dataset/mine_dev.csv",index=False)

df = pd.get_dummies(df, columns=["designation_tr","province","variety"])

df.columns

df = pd.read_csv("competition_dataset/mine_dev.csv")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    df["designation_tr"].value_counts()[:200]
    
len(df["designation_tr"].value_counts())

import gmaps
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html

locations = df[['lat', 'long']]
weights = df['quality']
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap_layer)
embed_minimal_html('export.html', views=[fig])

df["variety"].value_counts()[:50]

df["province"].value_counts()[:50]

df["region_complete"] = df["region_complete"].replace(r'^\s*$', np.NaN, regex=True)


df = pd.get_dummies(df, columns=["region_complete","variety"])

df.to_csv("competition_dataset/mine_dev_with_dummies.csv",index=False)

df = pd.read_csv("competition_dataset/mine_dev_with_dummies.csv")

len(df.columns)

from platform import python_version

print(python_version())


In [230]:
import pandas as pd
import numpy as np

In [231]:
#Caricamento di x
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

In [213]:
df_test = df.drop_duplicates()

In [215]:
df_test[df_test["province"] == "Idaho"]

,country,description,designation,province,region_1,region_2,variety,winery,quality
116,US,"The aromas are light and elusive, with the pal...",NaN,Idaho,NaN,NaN,Sangiovese,Williamson Vineyard,40.0
1107,US,"Elegantly styled, this offers notes of herbs, ...",Reserve,Idaho,NaN,NaN,Merlot,3 Horse Ranch Vineyards,45.0
1501,US,"Sweet, grapy, raspberry-candy fruit flavors ar...",Arena Valley Vineyard,Idaho,NaN,NaN,Sangiovese,Snake River,41.0
1622,US,"Aromas of green herbs, flowers and soil lead t...",NaN,Idaho,NaN,NaN,Petit Verdot,Colter's Creek,50.0
2168,US,"Friendly and exuberant, exhibiting flavors of ...",Three Vineyard Cuvée,Idaho,Idaho,NaN,Syrah,Koenig Vineyards,43.0
2950,US,"A little reticent on the nose, but opens up ge...",NaN,Idaho,Idaho,NaN,Riesling,Colter's Creek,26.0
3847,US,"A blend of fruit from Williamson, Bitner and H...",Three Vineyard Cuvée,Idaho,NaN,NaN,Syrah,Koenig Vineyards,42.0
4994,US,Aromas of reduction and overripe fruit are fol...,Estate Grown,Idaho,NaN,NaN,Pinot Gris,Sawtooth,31.0
5993,US,Blackberry and dark fruit jam on the nose and ...,Arena Valley Vineyard,Idaho,Idaho,NaN,Barbera,Snake River,41.0
6874,US,"Wilting white flowers, tangerine and jasmine a...",Special Harvest,Idaho,NaN,NaN,Riesling,Ste. Chapelle,27.0


In [208]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85028 entries, 0 to 120738
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      85025 non-null  object 
 1   description  85028 non-null  object 
 2   designation  59084 non-null  object 
 3   province     85025 non-null  object 
 4   region_1     71139 non-null  object 
 5   region_2     34294 non-null  object 
 6   variety      85028 non-null  object 
 7   winery       85028 non-null  object 
 8   quality      85028 non-null  float64
dtypes: float64(1), object(8)
memory usage: 6.5+ MB


In [149]:
mask = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull()) & df_test['province'].isnull()

df_test[mask].info()

# In 13889 record entrambe le region sono nulle
# In 3 record entrambe le region sono nulle e anche la province

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 28353 to 92096
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      0 non-null      object 
 1   description  3 non-null      object 
 2   designation  3 non-null      object 
 3   province     0 non-null      object 
 4   region_1     0 non-null      object 
 5   region_2     0 non-null      object 
 6   variety      3 non-null      object 
 7   winery       3 non-null      object 
 8   quality      3 non-null      float64
dtypes: float64(1), object(8)
memory usage: 240.0+ bytes


In [209]:
mask = df_test['region_1'].isnull() & df_test['region_2'].isnull()

province_with_region_nans = df_test[mask]["province"].unique()

print(len(province_with_region_nans))

df_test[df_test["province"] == "Mendoza Province"]

mask_3 = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull())
df_test[mask_3]

# In 13889 record entrambe le region sono nulle

387


,country,description,designation,province,region_1,region_2,variety,winery,quality
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0
5,New Zealand,Classic gooseberry and pink grapefruit notes f...,NaN,Marlborough,NaN,NaN,Sauvignon Blanc,Cloudy Bay,48.0
19,Chile,"Cedar, cassis and blueberry aromas open this m...",Catalina,Rapel Valley,NaN,NaN,Bordeaux-style Red Blend,Santa Ema,49.0
20,Austria,"With its 18 months wood aging, this is a velve...",Reserve,Carnuntum,NaN,NaN,Pinot Noir,Markowitsch,56.0
32,Chile,"Gold in color, this blend of 50% Riesling and ...",Late Harvest,Curicó Valley,NaN,NaN,White Blend,Montes,40.0
...,...,...,...,...,...,...,...,...,...
120684,Portugal,A soft and easy white wine that has a lightly ...,Periquita White,Península de Setúbal,NaN,NaN,Portuguese White,José Maria da Fonseca,29.0
120694,Chile,Snappy aromas of spring peas and fresh herbs c...,U Estate Grown,Maipo Valley,NaN,NaN,Sauvignon Blanc,Undurraga,26.0
120709,Chile,Lean aromas of bell pepper and green fruits ar...,Gran Reserva,Casablanca Valley,NaN,NaN,Sauvignon Blanc,Indomita,32.0
120725,New Zealand,"Toasty, but with the fruit to back it up. Peac...",Moutere,New Zealand,NaN,NaN,Chardonnay,Neudorf,59.0


In [210]:
mask_2 = (df_test['region_1'].isnull() & df_test['region_2'].isnull()) == False

df_test_at_least_one_region = df_test[mask_2]

province_with_at_least_one_region = df_test[mask_2]["province"].unique()

df_test_at_least_one_region[df_test_at_least_one_region["province"].isin(province_with_region_nans)]

country_with_at_least_one_region = df_test[mask_2]["country"].unique()

In [192]:
province_with_at_least_one_region

array(['Alsace', 'California', 'Oregon', 'Southern Italy', 'Tuscany',
       'Mendoza Province', 'Washington', 'Bordeaux', 'Champagne',
       'Languedoc-Roussillon', 'Northeastern Italy', 'Piedmont',
       'Provence', 'Sicily & Sardinia', 'Veneto', 'Central Italy',
       'Burgundy', 'South Australia', 'Catalonia', 'Rhône Valley',
       'Northern Spain', 'Victoria', 'France Other', 'Galicia',
       'New York', 'Western Australia', 'British Columbia',
       'Southwest France', 'Other', 'Central Spain', 'Australia Other',
       'Loire Valley', 'Virginia', 'Spanish Islands', 'Beaujolais',
       'Northwestern Italy', 'Tasmania', 'Ontario', 'Levante', 'Lombardy',
       'Pennsylvania', 'Ohio', 'New South Wales', 'Andalucia',
       'New Mexico', 'Idaho', 'Italy Other', 'Colorado', 'Arizona',
       'Spain Other', 'Vermont', 'Texas', 'New Jersey', 'North Carolina',
       'Missouri', 'Michigan', 'Massachusetts', 'Iowa', 'Canada Other',
       'Queensland', 'Nevada', 'Connecticut', 'Ke

In [229]:
df_test = df.drop_duplicates()

mask_2 = (df_test['region_1'].isnull() & df_test['region_2'].isnull()) == False
province_with_at_least_one_region = df_test[mask_2]["province"].unique()

mask4 = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull()) & (df_test['province'].isin(province_with_at_least_one_region))
df_test_no_region_in_at_least_one_prov = df_test[mask4].copy() # province che non hanno region in almeno un record e hanno region in almeno un record

province_interested = df_test_no_region_in_at_least_one_prov["province"].unique()

# In 88 record le due regioni sono nulle e la provincia ha almeno un record con una regione

mask = df_test["province"].isin(province_interested)
df_test_interested = df_test[mask].copy()

df_test_interested.loc[df_test_interested["region_1"] == df_test_interested["region_2"], "region_2"] = "" # se region1 e 2 sono uguali rendo region 2 una stringa vuota
df_test_interested["region_complete"] = df_test_interested[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df_test_interested["region_complete"] = df_test_interested["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

provinces = (df_test_interested[["province","region_complete"]].groupby(['province']).agg(lambda x:x.value_counts().index[0] if len(x.value_counts().index) > 0 else "Null").to_dict())

df_test_interested[((df_test_interested["region_1"].isnull() == True) & (df_test_interested["region_2"].isnull() == True))].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 116 to 119940
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country          88 non-null     object 
 1   description      88 non-null     object 
 2   designation      57 non-null     object 
 3   province         88 non-null     object 
 4   region_1         0 non-null      object 
 5   region_2         0 non-null      object 
 6   variety          88 non-null     object 
 7   winery           88 non-null     object 
 8   quality          88 non-null     float64
 9   region_complete  0 non-null      object 
dtypes: float64(1), object(9)
memory usage: 7.6+ KB


In [167]:
mask6 = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull()) & (df_test['country'].isin(country_with_at_least_one_region))
df_test_no_region_in_at_least_one_coun = df_test[mask6]

df_test_no_region_in_at_least_one_prov.info()

# In 88 record le due regioni sono nulle e la nazione ha almeno un record con una regione

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 116 to 119940
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      88 non-null     object 
 1   description  88 non-null     object 
 2   designation  57 non-null     object 
 3   province     88 non-null     object 
 4   region_1     0 non-null      object 
 5   region_2     0 non-null      object 
 6   variety      88 non-null     object 
 7   winery       88 non-null     object 
 8   quality      88 non-null     float64
dtypes: float64(1), object(8)
memory usage: 6.9+ KB


In [139]:
mask5 = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull()) & (df_test['province'].isnull())

df_test_no_region_no_prov = df_test[mask5]

df_test_no_region_no_prov.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 28353 to 92096
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      0 non-null      object 
 1   description  3 non-null      object 
 2   designation  3 non-null      object 
 3   province     0 non-null      object 
 4   region_1     0 non-null      object 
 5   region_2     0 non-null      object 
 6   variety      3 non-null      object 
 7   winery       3 non-null      object 
 8   quality      3 non-null      float64
dtypes: float64(1), object(8)
memory usage: 240.0+ bytes


In [32]:
df

,,country,description,designation,region_2,variety,winery,quality
province,region_1,,,,,,,
Alsace,Crémant d'Alsace,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,NaN,Pinot Blanc,Lucien Albrecht,45.0
California,Paso Robles,US,"Simple and dry, this Cabernet has modest black...",NaN,Central Coast,Cabernet Sauvignon,Castle Rock,31.0
Oregon,Willamette Valley,US,"This lovely wine captures the floral, perfumed...",NaN,Willamette Valley,Gewürztraminer,Château Bianca,35.0
Alentejano,NaN,Portugal,"The aromas are the thing here, as so often wit...",NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0
Southern Italy,Pompeiano,Italy,"This is an interesting, outright strange wine ...",Natì,NaN,Coda di Volpe,Sorrentino,37.0
...,...,...,...,...,...,...,...,...
Northeastern Italy,Friuli,Italy,This is a lovely Sauvignon with very character...,NaN,NaN,Sauvignon,Plozner,40.0
Northern Spain,Rioja,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,NaN,Tempranillo Blend,Ramón Bilbao,41.0
South Australia,Adelaide Hills,Australia,Surprisingly complete and harmonious for a Rie...,NaN,NaN,Riesling,Wagtail,26.0


In [261]:
import math 

# Preprocessing su x
df = df.drop_duplicates() # Rimuovo duplicati

df = df.drop(df[df["quality"] == 0].index) #Rimuovo righe con qualità uguale a zero ( sono solo 15 in totale )
df.reset_index(drop=True, inplace=True)

df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","réserve"],["reserve","reserve","reserve"])


df.loc[df["region_1"] == df["region_2"], "region_2"] = "" # se region1 e 2 sono uguali rendo region 2 una stringa vuota
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df["region_complete"] = df["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

df["description_len"] = df["description"].map(len)

# --------------------------------------------
# Riempio alcune region_complete nulle


mask_1 = (df['region_1'].isnull() & df['region_2'].isnull()) == False
province_with_at_least_one_region = df[mask_1]["province"].unique()

mask_2 = (df['region_1'].isnull()) & (df['region_2'].isnull()) & (df['province'].isin(province_with_at_least_one_region))
df_no_region_in_at_least_one_prov = df[mask_2].copy() # province che non hanno region in almeno un record e hanno region in almeno un record

province_interested = df_no_region_in_at_least_one_prov["province"].unique()

# In 88 record le due regioni sono nulle e la provincia ha almeno un record con una regione

mask = df["province"].isin(province_interested)
df_interested = df[mask].copy()

province_region = (df_interested[["province","region_complete"]].groupby(['province']).agg(lambda x:x.value_counts().index[0] if len(x.value_counts().index) > 0 else "Null").to_dict())

province_region = province_region["region_complete"]

def change_region_complete(x):    
    return province_region[x["province"]] if (type(x["region_complete"]) == float and math.isnan(x["region_complete"])) else x["region_complete"]

df.loc[mask, "region_complete"] = df.loc[mask].apply(change_region_complete, axis=1) 

# --------------------------------------------

df["region_complete"].fillna('None', inplace=True)
df["variety"].fillna('None', inplace=True)
df["country"].fillna('None', inplace=True)
df["winery"].fillna('None', inplace=True)
df["province"].fillna('None', inplace=True)
df["designation_tr"].fillna('None', inplace=True)

{'region_complete': {'Idaho': 'Idaho  ', 'Mendoza Province': 'Mendoza  ', 'Provence': 'Côtes de Provence  ', 'Rhône Valley': 'Côtes du Rhône  ', 'Southwest France': 'Cahors  '}}
{'Idaho': 'Idaho  ', 'Mendoza Province': 'Mendoza  ', 'Provence': 'Côtes de Provence  ', 'Rhône Valley': 'Côtes du Rhône  ', 'Southwest France': 'Cahors  '}
         country                                        description  \
11     Argentina  This is a straightforward, regular Cab with be...   
13     Argentina  A bit stalky on the nose, with tropical aromas...   
21     Argentina  Floral and tropical aromas are a direct reflec...   
41        France  A fresh rosé, so crisp, bright and lively. Thi...   
46     Argentina  Raspy cherry aromas precede a scratchy palate ...   
...          ...                                                ...   
84910     France  Restrained in style, with modest pear and hone...   
84939  Argentina  Opens with grassy notes along with cleaner, fr...   
84944     France  From the

In [24]:
df["designation_tr"].value_counts()

None                                  25939
reserve                                3081
barrel sample                          1151
estate                                  880
brut                                    339
                                      ...  
the maneater derby vineyard               1
malombra                                  1
herrerias vineyard                        1
les fiefs de grenouilles grand cru        1
mairlant                                  1
Name: designation_tr, Length: 27632, dtype: int64

In [3]:
# Preprocessing su x_eval

df_eval = pd.read_csv("competition_dataset/eval.tsv", sep="\t")

df_eval["designation_tr"] = df_eval["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df_eval["designation_tr"] = df_eval["designation_tr"].replace(["reserva","riserva","réserve"],["reserve","reserve","reserve"])

df_eval.loc[df_eval["region_1"] == df_eval["region_2"], "region_2"] = ""
df_eval["region_complete"] = df_eval[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df_eval["region_complete"] = df_eval["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

df_eval["description_len"] = df_eval["description"].map(len)

mask = df_eval["province"].isin(province_interested)
df_eval.loc[mask, "region_complete"] = df_eval.loc[mask].apply(change_region_complete, axis=1) 


df_eval["region_complete"].fillna('None', inplace=True)
df_eval["variety"].fillna('None', inplace=True)
df_eval["country"].fillna('None', inplace=True)
df_eval["winery"].fillna('None', inplace=True)
df_eval["province"].fillna('None', inplace=True)
df_eval["designation_tr"].fillna('None', inplace=True)

In [4]:
#Aggiungo a x i bit delle winery con più vini

from sklearn.preprocessing import OneHotEncoder

df_group_by_winery = df[["winery","quality"]].groupby(by="winery",sort=True).agg(['mean', 'count']).sort_values(by=("quality","count"),ascending=False)


top_N_winery = df_group_by_winery[df_group_by_winery[("quality","count")] >= 20].index.values

df.loc[~df["winery"].isin(top_N_winery) ,"winery"] = "None"

ohc_w = OneHotEncoder(handle_unknown="ignore")

ohWinery = ohc_w.fit_transform(df["winery"].values.reshape(-1,1)).toarray()
dfOneHot_w = pd.DataFrame(ohWinery, columns=["winery_" + str(ohc_w.categories_[0][i]) for i in range(len(ohc_w.categories_[0]))])

df = pd.concat([df,dfOneHot_w], axis=1)

In [18]:
top_N_winery

array(['Williams Selyem', 'Testarossa', 'DFJ Vinhos',
       'Chateau Ste. Michelle', 'Columbia Crest', 'Bouchard Père & Fils',
       'Concha y Toro', 'Kendall-Jackson', 'Joseph Drouhin', 'Trapiche',
       'De Loach', 'Kenwood', 'Georges Duboeuf', 'Albert Bichot',
       'Wines & Winemakers', 'Cameron Hughes', 'Robert Mondavi',
       'Louis Latour', 'Errazuriz', 'Martin Ray', 'Concannon',
       'Dry Creek Vineyard', 'Olivier Leflaive', 'Morgan',
       "L'Ecole No. 41", "D'Arenberg", 'Waterbrook', 'Iron Horse',
       'Calera', 'Yalumba', 'Bodega Norton', 'Hogue', 'Santa Rita',
       'Gary Farrell', 'Sineann', 'Renwood', 'Montes', 'Rodney Strong',
       'Wellington', 'Seven Hills', 'Saint Clair', 'Planeta', 'Novy',
       'Viu Manent', 'Bründlmayer', 'Louis Jadot', 'J. Lohr',
       'Thomas Fogarty', 'Santa Ema', 'Adelaida', 'Dão Sul', 'Brander',
       'Tamarack Cellars', 'Henri de Villamont', 'Dutton-Goldfield',
       'Fess Parker', 'Barton & Guestier', 'Penfolds', 'Lynmar', '

In [19]:
df["province"].value_counts()

California                         35564
Washington                          7818
Tuscany                             5843
Bordeaux                            4866
Northern Spain                      3930
                                   ...  
San Antonio de las Minas Valley        1
Sterea Ellada                          1
Lemesos                                1
Lemnos                                 1
Langenlois                             1
Name: province, Length: 444, dtype: int64

In [5]:
#Aggiungo a x_eval i bit delle winery con più vini

from sklearn.preprocessing import OneHotEncoder

ohWinery = ohc_w.transform(df_eval["winery"].values.reshape(-1,1)).toarray()
dfOneHot_w = pd.DataFrame(ohWinery, columns=["winery_" + str(ohc_w.categories_[0][i]) for i in range(len(ohc_w.categories_[0]))])

df_eval = pd.concat([df_eval,dfOneHot_w], axis=1)

In [ ]:
df

In [ ]:
df.boxplot(column=['quality'])

#Aggiungo coordinate lat long a x
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, how="left", on="region_complete")

print(df.columns)
lat_mean = df['lat'].mean()
long_mean = df['long'].mean()

df["lat"].fillna(lat_mean, inplace=True)
df["long"].fillna(long_mean, inplace=True)


#Aggiungo coordinate lat long a x_eval
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df_eval = pd.merge(df_eval, df_region_coordinates, how="left", on="region_complete")

In [6]:
# Aggiungo a x bit di encoding di region_complete e variety e country e province
from sklearn.preprocessing import OneHotEncoder

ohc_r = OneHotEncoder(handle_unknown="ignore")
ohc_v = OneHotEncoder(handle_unknown="ignore")
ohc_c = OneHotEncoder(handle_unknown="ignore")
ohc_p = OneHotEncoder(handle_unknown="ignore")

ohRegion = ohc_r.fit_transform(df["region_complete"].values.reshape(-1,1)).toarray()
ohVariety = ohc_v.fit_transform(df["variety"].values.reshape(-1,1)).toarray()
ohCountry = ohc_c.fit_transform(df["country"].values.reshape(-1,1)).toarray()
ohProvince = ohc_p.fit_transform(df["province"].values.reshape(-1,1)).toarray()

dfOneHot_r = pd.DataFrame(ohRegion, columns=["region_complete_" + str(ohc_r.categories_[0][i]) for i in range(len(ohc_r.categories_[0]))])
dfOneHot_v = pd.DataFrame(ohVariety, columns=["variety_" + str(ohc_v.categories_[0][i]) for i in range(len(ohc_v.categories_[0]))])
dfOneHot_c = pd.DataFrame(ohCountry, columns=["country_" + str(ohc_c.categories_[0][i]) for i in range(len(ohc_c.categories_[0]))])
dfOneHot_p = pd.DataFrame(ohProvince, columns=["province_" + str(ohc_p.categories_[0][i]) for i in range(len(ohc_p.categories_[0]))])

df = pd.concat([df,dfOneHot_r,dfOneHot_v, dfOneHot_c, dfOneHot_p], axis=1)

In [7]:
# Aggiungo a x_eval bit di encoding di region_complete e variety e country e province

ohRegion = ohc_r.transform(df_eval["region_complete"].values.reshape(-1,1)).toarray()
ohVariety = ohc_v.transform(df_eval["variety"].values.reshape(-1,1)).toarray()
ohCountry = ohc_c.transform(df_eval["country"].values.reshape(-1,1)).toarray()
ohProvince = ohc_p.transform(df_eval["province"].values.reshape(-1,1)).toarray()


dfOneHot_r = pd.DataFrame(ohRegion, columns=["region_complete_" + str(ohc_r.categories_[0][i]) for i in range(len(ohc_r.categories_[0]))])
dfOneHot_v = pd.DataFrame(ohVariety, columns=["variety_" + str(ohc_v.categories_[0][i]) for i in range(len(ohc_v.categories_[0]))])
dfOneHot_c = pd.DataFrame(ohCountry, columns=["country_" + str(ohc_c.categories_[0][i]) for i in range(len(ohc_c.categories_[0]))])
dfOneHot_p = pd.DataFrame(ohProvince, columns=["province_" + str(ohc_p.categories_[0][i]) for i in range(len(ohc_p.categories_[0]))])

df_eval = pd.concat([df_eval,dfOneHot_r,dfOneHot_v,dfOneHot_c,dfOneHot_p], axis=1)

print(df.shape, df_eval.shape)

(120729, 3838) (30186, 3837)


df_group_by_designation_tr = df[["designation_tr","quality"]].groupby(by="designation_tr",sort=True).agg(['mean','count'])
df_group_by_designation_tr.reset_index()
df_group_by_designation_tr = df_group_by_designation_tr[df_group_by_designation_tr[("quality","count")] >= 10]

df_group_by_designation_tr.drop(["None"], inplace=True)

df_group_by_designation_tr.columns = df_group_by_designation_tr.columns.droplevel(0)
df_group_by_designation_tr.drop(columns=["count"], inplace=True)
df_group_by_designation_tr.columns = ["designation_tr_quality_mean"]

df_group_by_designation_tr
#df = pd.merge(df, df_group_by_designation_tr, how="left", on="designation_tr")

df_test = pd.merge(df, df_group_by_designation_tr, how="left", on="designation_tr")

df_test["designation_tr_quality_mean"].fillna(-1, inplace=True)

# Aggiungo a x i chunks delle designation
from sklearn.preprocessing import OneHotEncoder

df_group_by_designation_tr = df[["designation_tr","quality"]].groupby(by="designation_tr",sort=True).agg(['mean','count'])
df_group_by_designation_tr.reset_index()
df_group_by_designation_tr = df_group_by_designation_tr[df_group_by_designation_tr[("quality","count")] >= 10]

df_group_by_designation_tr.drop(["None"], inplace=True)

df_group_by_designation_tr.columns = df_group_by_designation_tr.columns.droplevel(0)
df_group_by_designation_tr.drop(columns=["count"], inplace=True)
df_group_by_designation_tr.columns = ["desi_quality_mean"]


df = pd.merge(df, df_group_by_designation_tr, how="left", on="designation_tr")
df["desi_quality_mean"].fillna(-1, inplace=True)
df["desi_quality_mean"] = df["desi_quality_mean"].astype(int)

ohc_d = OneHotEncoder(handle_unknown="ignore")
ohDesignation_quality_mean = ohc_d.fit_transform(df["desi_quality_mean"].values.reshape(-1,1)).toarray()

dfOneHot_dq = pd.DataFrame(ohDesignation_quality_mean, columns=["desi_quality_mean_" + str(ohc_d.categories_[0][i]) for i in range(len(ohc_d.categories_[0]))])

df.drop(columns=["desi_quality_mean"], inplace=True)
 
df = pd.concat([df,dfOneHot_dq], axis=1)

# Aggiungo a x_eval i chunks delle designation
from sklearn.preprocessing import OneHotEncoder

df_eval = pd.merge(df_eval, df_group_by_designation_tr, how="left", on="designation_tr")

df_eval["desi_quality_mean"] = df_eval["desi_quality_mean"].fillna(-1)

ohDesignation_quality_mean = ohc_d.transform(df_eval["desi_quality_mean"].values.reshape(-1,1)).toarray()

dfOneHot_dq = pd.DataFrame(ohDesignation_quality_mean, columns=["desi_quality_mean_" + str(ohc_d.categories_[0][i]) for i in range(len(ohc_d.categories_[0]))])

df_eval.drop(columns=["desi_quality_mean"], inplace=True)

df_eval = pd.concat([df_eval,dfOneHot_dq], axis=1)

#Aggiungo i bit dei primi N designation a x
from sklearn.preprocessing import OneHotEncoder

N = 300 
top_N_designation_tr = list(df["designation_tr"].value_counts()[1:N+1].keys()) #non includo None values

df.loc[~df["designation_tr"].isin(top_N_designation_tr) ,"designation_tr"] = "None"

ohc_d = OneHotEncoder(handle_unknown="ignore")
ohDesignation_tr = ohc_d.fit_transform(df["designation_tr"].values.reshape(-1,1)).toarray()

dfOneHot_d = pd.DataFrame(ohDesignation_tr, columns=["designation_tr_" + str(ohc_d.categories_[0][i]) for i in range(len(ohc_d.categories_[0]))])


df = pd.concat([df,dfOneHot_d], axis=1)

#Aggiungo i bit dei primi N designation a x eval
from sklearn.preprocessing import OneHotEncoder

N = 300
ohDesignation = ohc_d.transform(df_eval["designation_tr"].values.reshape(-1,1)).toarray()
dfOneHot_d = pd.DataFrame(ohDesignation, columns=["designation_tr_" + str(ohc_d.categories_[0][i]) for i in range(len(ohc_d.categories_[0]))])

df_eval = pd.concat([df_eval,dfOneHot_d], axis=1)


In [11]:
#Creo Vectorizer per x e x_eval
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english")

wpm = vectorizer.fit_transform(df["description"].fillna(""))

In [12]:
#Aggiungo a x i bit delle N parole più frequenti nella description 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", binary=True, use_idf=False,norm=False)

wpm = vectorizer.fit_transform(df["description"].fillna(""))

N = 1000
freq = sorted(zip(vectorizer.get_feature_names(), wpm.sum(axis=0).tolist()[0]),key=lambda x: x[1], reverse=True)[:N]

words = [ word for word, _ in freq ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
df_words = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df.index)

df = pd.concat([df,df_words], axis=1)

In [13]:
#Aggiungo a x_eval i bit delle N parole più frequenti nella description 
from sklearn.feature_extraction.text import TfidfVectorizer

wpm = vectorizer.transform(df_eval["description"].fillna(""))


df_words_eval = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df_eval.index)

df_eval = pd.concat([df_eval,df_words_eval], axis=1)


#Test tdidf description su x
from sklearn.feature_extraction.text import TfidfVectorizer

N = 500
vectorizer = TfidfVectorizer(stop_words="english", use_idf=True, max_features = N)

wpm = vectorizer.fit_transform(df["description"].fillna(""))

words = [ word for word, _ in freq ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
df_words = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df.index)

df_words

In [14]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, n_jobs=-1,max_features="sqrt")

In [15]:
df.iloc[:,9:]

,designation_tr,region_complete,description_len,winery_3 Horse Ranch Vineyards,winery_:Nota Bene,winery_A to Z,winery_Abacela,winery_Abadia Retuerta,winery_Abbazia Santa Anastasia,winery_Abbazia di Novacella,...,word_yeast,word_yeasty,word_yellow,word_young,word_youth,word_youthful,word_zest,word_zesty,word_zin,word_zinfandel
0,None,Crémant d'Alsace,184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,None,Paso Robles Central Coast,133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,None,Willamette Valley,202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,None,None,210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,None,Pompeiano,237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120724,None,Friuli,323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120725,reserve,Rioja,237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120726,None,Adelaide Hills,237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120727,spätlese,None,252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Test PCA 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df.iloc[:, 11:]
y = df["quality"]

scaler = MinMaxScaler()
X_rescaled = scaler.fit_transform(X)
#y_rescaled = scaler.fit_transform(y)

pca = PCA(n_components = 0.99)
pca.fit(X_rescaled)
reduced = pca.transform(X_rescaled)

reduced


In [ ]:
reduced.shape

#Predict con PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
            
X_train, X_test, y_train, y_test = train_test_split(reduced, y, test_size=0.25, random_state=34)

regr.fit(X_train,y_train)
            
y_pred = regr.predict(X_test)

r2_score(y_test, y_pred)

In [47]:
X.shape

(120729, 4827)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df.iloc[:, 11:]
y = df["quality"]
            
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=34)

regr.fit(X_train,y_train)
            
y_pred = regr.predict(X_test)

r2_score(y_test, y_pred)

0.7178835729545676

In [18]:
#prova
sorted(zip(list(X.columns[11:]), regr.feature_importances_), key=lambda x: x[1],reverse=True)

[('winery_Acorn', 0.03621998462328643),
 ('region_complete_Capay Valley Central Valley', 0.014681328978429322),
 ('variety_Portuguese White', 0.01097016365066633),
 ('word_term', 0.010026398334819088),
 ('word_river', 0.00785666853839371),
 ('country_Hungary', 0.0074757402164872455),
 ('region_complete_Offida Passerina  ', 0.006496698938192226),
 ('province_Casablanca Valley', 0.00641361141331494),
 ('word_prune', 0.006137537546168984),
 ('province_Alentejano', 0.005813051290870432),
 ('province_Colonia', 0.005693322444533018),
 ('province_Cederberg', 0.005061329268910912),
 ('region_complete_Napa-Sonoma-Monterey North Coast', 0.004915258287770199),
 ('word_anise', 0.004842048554429149),
 ('word_washington', 0.0047811249497002675),
 ('word_worth', 0.00466244602867315),
 ('country_Czech Republic', 0.00452265720326254),
 ('word_blueberry', 0.004503050333028949),
 ('word_colored', 0.004469486117422337),
 ('region_complete_Chianti  ', 0.004442369144265432),
 ('region_complete_Missouri  ', 

In [ ]:
#Ricerca con grid search

from sklearn.model_selection import GridSearchCV

X = df.iloc[:, 11:]
y = df["quality"]
            
param_grid = {
"n_estimators": [100, 250],
"criterion": ["mse"],
"max_features": ["auto", "sqrt", "log2"],
"random_state": [42], # always use the samet random seed
"n_jobs": [-1], # for parallelization
}

gs = GridSearchCV(RandomForestRegressor(), param_grid, scoring="r2", n_jobs=-1,cv=5)
gs.fit(X, y)
gs.best_score_

In [ ]:
from prettytable import PrettyTable

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

def evaluate_model(X, y, model, model_name):

    X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.25,random_state=42,shuffle=True)
    # plot the real function and the training points
    LW = 2
    #fig, ax = plt.subplots()
    #ax.plot(X, y, color='cornflowerblue', linewidth=.5*LW, label="ground truth")
    #ax.scatter(X_train, y_train, color='navy', s=30, marker='o',label="training points")
    # predict the test points and plot them onto the chart
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    #ax.plot(X_test, y_hat, linewidth=LW, label=name, color='r')
    #fig.suptitle(f"{f} approximated by {model_name}")
    #fig.legend()
    return r2_score(y_test, y_hat)



degree = 2
models = [
    LinearRegression(),
    Ridge(random_state=42),
    RandomForestRegressor(n_estimators=100,n_jobs=-1,max_features="sqrt"),
    make_pipeline(
        make_column_transformer(
            (FunctionTransformer(np.sin), [0]),
            (PolynomialFeatures(degree), [0])
        ),
        LinearRegression()
    ),
    make_pipeline(
        make_column_transformer(
            (FunctionTransformer(np.sin), [0]),
            (PolynomialFeatures(degree), [0])
        ),
        Ridge(alpha=1)
    )
]

names = [
    'linreg',
    'ridge',
    'rf',
    f'sin+poly{degree}+linreg',
    f'sin+poly{degree}+ridge'
]


t = PrettyTable()
t.field_names = ['model', 'R2']
X = df.iloc[:, 11:]
y = df["quality"]
for model, name in zip(models, names):
    print(model, name)
    r2 = evaluate_model(X, y, model, name)
    t.add_row([name, r2])
print(t)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

X = df.iloc[:, 11:]
y = df["quality"]

regr_rf = RandomForestRegressor()
parameters_rf = {'n_estimators':[50,100], 'n_jobs':[-1], "max_features":["sqrt","log2"], "random_state":[34]}
clf_rf = GridSearchCV(regr_rf, parameters_rf, scoring="r2",verbose=3)

clf_rf.fit(X,y)
print(clf_rf.best_params_)
print(clf_rf.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

X = df.iloc[:, 11:]
y = df["quality"]

regr_rf = Ridge()
parameters_rf = { "alpha":np.arange(0.1, 1.1, 0.2), "fit_intercept":[True,False],"random_state":[34]}
clf_rf = GridSearchCV(regr_rf, parameters_rf, scoring="r2",verbose=3)

clf_rf.fit(X,y)
print(clf_rf.best_params_)
print(clf_rf.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

X = df.iloc[:, 11:]
y = df["quality"]

regr_rf = Lasso()
parameters_rf = { "alpha":np.arange(0.1, 1.1, 0.2), "fit_intercept":[True,False],"random_state":[34]}
clf_rf = GridSearchCV(regr_rf, parameters_rf, scoring="r2",verbose=3)

clf_rf.fit(X,y)
print(clf_rf.best_params_)
print(clf_rf.best_score_)

In [ ]:
freq[:100]

In [ ]:
X.shape

In [ ]:
sorted(zip(list(X.columns[11:]), regr.feature_importances_), key=lambda x: x[1],reverse=True)

In [ ]:
X_train["lat"].isnull().sum()

In [ ]:
list(set(list(df_eval.iloc[:,10:].columns)) - set(list(df.iloc[:, 11:].columns)))

In [ ]:
df_eval.drop(columns=["desi_quality_mean"], inplace=True)


In [ ]:
df_eval.iloc[:,10:]

In [ ]:
df.iloc[:, 11:]

In [ ]:
X = df.iloc[:, 11:]
y = df["quality"]
            
regr.fit(X,y)

In [ ]:
X_eval = df_eval.iloc[:,10:]

y_eval = regr.predict(X_eval)

In [ ]:
y_eval

In [ ]:
with open("my_submission.csv","w") as f:
    f.write("Id,Predicted\n")
    for i,result in enumerate(y_eval):
        f.write(str(i) + "," + str(y_eval[i])+"\n")